In [4]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from sqlalchemy import *
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from flask import Flask, jsonify

In [5]:
engine = create_engine("sqlite:///Resources/belly_button_biodiversity.sqlite", echo=False)

In [6]:
inspector = inspect(engine)
inspector.get_table_names()

['otu', 'samples', 'samples_metadata']

In [7]:
columns = inspector.get_columns('otu')
for c in columns:
    print(c['name'], c["type"])

otu_id INTEGER
lowest_taxonomic_unit_found TEXT


In [8]:
columns = inspector.get_columns('samples')
for c in columns:
    print(c['name'], c["type"])

otu_id INTEGER
BB_940 INTEGER
BB_941 INTEGER
BB_943 INTEGER
BB_944 INTEGER
BB_945 INTEGER
BB_946 INTEGER
BB_947 INTEGER
BB_948 INTEGER
BB_949 INTEGER
BB_950 INTEGER
BB_952 INTEGER
BB_953 INTEGER
BB_954 INTEGER
BB_955 INTEGER
BB_956 INTEGER
BB_958 INTEGER
BB_959 INTEGER
BB_960 INTEGER
BB_961 INTEGER
BB_962 INTEGER
BB_963 INTEGER
BB_964 INTEGER
BB_966 INTEGER
BB_967 INTEGER
BB_968 INTEGER
BB_969 INTEGER
BB_970 INTEGER
BB_971 INTEGER
BB_972 INTEGER
BB_973 INTEGER
BB_974 INTEGER
BB_975 INTEGER
BB_978 INTEGER
BB_1233 INTEGER
BB_1234 INTEGER
BB_1235 INTEGER
BB_1236 INTEGER
BB_1237 INTEGER
BB_1238 INTEGER
BB_1242 INTEGER
BB_1243 INTEGER
BB_1246 INTEGER
BB_1253 INTEGER
BB_1254 INTEGER
BB_1258 INTEGER
BB_1259 INTEGER
BB_1260 INTEGER
BB_1264 INTEGER
BB_1265 INTEGER
BB_1273 INTEGER
BB_1275 INTEGER
BB_1276 INTEGER
BB_1277 INTEGER
BB_1278 INTEGER
BB_1279 INTEGER
BB_1280 INTEGER
BB_1281 INTEGER
BB_1282 INTEGER
BB_1283 INTEGER
BB_1284 INTEGER
BB_1285 INTEGER
BB_1286 INTEGER
BB_1287 INTEGER
BB_1288 IN

In [9]:
columns = inspector.get_columns('samples_metadata')
for c in columns:
    print(c['name'], c["type"])

SAMPLEID INTEGER
EVENT TEXT
ETHNICITY TEXT
GENDER TEXT
AGE INTEGER
WFREQ INTEGER
BBTYPE TEXT
LOCATION TEXT
COUNTRY012 TEXT
ZIP012 INTEGER
COUNTRY1319 TEXT
ZIP1319 INTEGER
DOG TEXT
CAT TEXT
IMPSURFACE013 INTEGER
NPP013 FLOAT
MMAXTEMP013 FLOAT
PFC013 FLOAT
IMPSURFACE1319 INTEGER
NPP1319 FLOAT
MMAXTEMP1319 FLOAT
PFC1319 FLOAT


In [23]:
engine.execute('SELECT * FROM otu LIMIT 10').fetchall()


[(1, 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus'),
 (2, 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus'),
 (3, 'Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Natronorubrum'),
 (4, 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'),
 (5, 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'),
 (6, 'Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter'),
 (7, 'Bacteria'),
 (8, 'Bacteria'),
 (9, 'Bacteria'),
 (10, 'Bacteria')]

In [11]:
engine.execute('SELECT * FROM samples LIMIT 10').fetchall()

[(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
engine.execute('SELECT * FROM samples_metadata LIMIT 10').fetchall()

[(940, 'BellyButtonsScienceOnline', 'Caucasian', 'F', 24, 2, 'I', 'Beaufort/NC', 'usa', 22306, 'usa', 22306, 'no', 'no', 8852, 37.17222214, 54.5, 0.0, 1, 0.0, 33.99000168, 25.5),
 (941, '0', 'Caucasian/Midleastern', 'F', 34, 1, 'I', 'Chicago/IL', '0', 0, '0', 0, '0', '0', 0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0),
 (943, 'BellyButtonsScienceOnline', 'Caucasian', 'F', 49, 1, 'I', 'Omaha/NE', '0', 0, '0', 0, '0', '0', 0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0),
 (944, 'BellyButtonsScienceOnline', 'European', 'M', 44, 1, 'I', 'NewHaven/CT', 'usa', 7079, 'usa', 8822, 'no', 'yes', 0, 35.81666565, 16.0, 0.0, 0, 6567.0, 32.40333176, 28.5),
 (945, 'BellyButtonsScienceOnline', 'Caucasian', 'F', 48, 1, 'I', 'Philidelphia/PA', 'usa', 84404, 'usa', 96025, 'no', 'no', 0, 37.78333282, 4.0, 0.0, 0, 5613.0, 33.63444519, 24.0),
 (946, 'BellyButtonsScienceOnline', 'Caucasian', 'F', 42, 3, 'I', 'Deerfield/MA', 'usa', 12538, 'usa', 12538, 'no', 'yes', 0, 34.76111221, 15.5, 0.0, 0, 6273.0, 29.40333366, 67.5),
 (947, 'B

In [13]:
metadata = MetaData()

In [17]:
metadata.reflect(engine, only=['otu', 'samples', 'samples_metadata'])

In [18]:
Base = automap_base(metadata=metadata)
Base.prepare()

In [19]:
otu, samples, samples_metadata = Base.classes.otu, Base.classes.samples, Base.classes.samples_metadata

In [20]:
session = Session(engine)

In [56]:
foo = session.query().limit(10)
for x in foo:
    print(x.otu_id, x.lowest_taxonomic_unit_found)

1 Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus
2 Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus
3 Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Natronorubrum
4 Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter
5 Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter
6 Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter
7 Bacteria
8 Bacteria
9 Bacteria
10 Bacteria


In [59]:
result = session.query(samples_metadata.SAMPLEID).all()

In [63]:
[ "BB_"+str(id[0]) for id in result ]

['BB_940',
 'BB_941',
 'BB_943',
 'BB_944',
 'BB_945',
 'BB_946',
 'BB_947',
 'BB_948',
 'BB_949',
 'BB_950',
 'BB_952',
 'BB_953',
 'BB_954',
 'BB_955',
 'BB_956',
 'BB_958',
 'BB_959',
 'BB_960',
 'BB_961',
 'BB_962',
 'BB_963',
 'BB_964',
 'BB_966',
 'BB_967',
 'BB_968',
 'BB_969',
 'BB_970',
 'BB_971',
 'BB_972',
 'BB_973',
 'BB_974',
 'BB_975',
 'BB_978',
 'BB_1233',
 'BB_1234',
 'BB_1235',
 'BB_1236',
 'BB_1237',
 'BB_1238',
 'BB_1242',
 'BB_1243',
 'BB_1246',
 'BB_1253',
 'BB_1254',
 'BB_1258',
 'BB_1259',
 'BB_1260',
 'BB_1264',
 'BB_1265',
 'BB_1273',
 'BB_1275',
 'BB_1276',
 'BB_1277',
 'BB_1278',
 'BB_1279',
 'BB_1280',
 'BB_1281',
 'BB_1282',
 'BB_1283',
 'BB_1284',
 'BB_1285',
 'BB_1286',
 'BB_1287',
 'BB_1288',
 'BB_1289',
 'BB_1290',
 'BB_1291',
 'BB_1292',
 'BB_1293',
 'BB_1294',
 'BB_1295',
 'BB_1296',
 'BB_1297',
 'BB_1298',
 'BB_1308',
 'BB_1309',
 'BB_1310',
 'BB_1374',
 'BB_1415',
 'BB_1439',
 'BB_1441',
 'BB_1443',
 'BB_1486',
 'BB_1487',
 'BB_1489',
 'BB_1490',
 

In [64]:
        result = session.query(samples_metadata).filter(samples_metadata.SAMPLEID == int("1265"))
    

In [65]:
result

In [82]:
row = result[0].__dict__
row.pop('_sa_instance_state', None)
print(row)
    

{'SAMPLEID': 1265, 'MMAXTEMP013': 31.5, 'COUNTRY012': 'usa', 'EVENT': 'BellyButtonsMuseum2', 'PFC013': 0.0, 'ZIP012': 27615, 'IMPSURFACE1319': 0, 'ETHNICITY': 'Caucasian', 'COUNTRY1319': 'n/a', 'GENDER': 'f', 'NPP1319': 0.0, 'ZIP1319': 'n/a', 'MMAXTEMP1319': 0.0, 'AGE': 10, 'DOG': 'yes', 'WFREQ': 0, 'PFC1319': 0.0, 'CAT': 'yes', 'BBTYPE': 'i', 'IMPSURFACE013': 0, 'LOCATION': 'Raleigh,NC', 'NPP013': 34.54111099}
